# Surface water

In [ ]:
import matplotlib
import nlmod
# add a logger with nice coloring
logger = nlmod.util.get_color_logger("INFO");

First we specify the model extent. This extent is specified by 4 numbers: [xmin, xmax, ymin, ymax].

In [ ]:
extent = [116_500, 120_000, 439_000, 442_000]

## Download BGT
Download the surface water items from the Basisregistratie Grootschalige Topografie (BGT: registration large-scale topography) to get the location of the surface water, contained in a GeoDataFrame.

In [ ]:
bgt = nlmod.read.bgt.get_bgt(extent)

Exercise 1: look at the contents of this GeoDataFrame. What kind of data is this: point-, line- or polygon-data?

## Plot BGT
Plot this GeoDataFrame. First create a figure and ax using `f, ax = nlmod.plot.get_map(extent)`, then plot the GeoDataFrame `bgt` using it's plot method, adding it to the created `ax`.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot(ax=ax);

## Plot column 'bronhouder'
As this GeoDataFrame does not contain any information about the stage of the surface water, we need to collect more data about these surface water bodies.

Exercise 2: plot the data-owner of each feature, which is contained in the column 'bronhouder' of the GeoDataFrame. Add "bronhouder" as the first argument of `bgt.plot()`, and make sure to plot a legend, by passing `legend=True`.

## Download stages from water boards
For all data-owners starting with a 'W' (from Waterboard), we download the level areas, which contain a summer- and a winter- stage. Use the method `nlmod.gwf.surface_water.download_level_areas`, with arguments `bgt` and `extent`.

In [ ]:
la = nlmod.gwf.surface_water.download_level_areas(bgt, extent=extent, raise_exceptions=False)

Exercise 3: What type of data does this method return: point-, line- or polygon-data?

## Plot summer stage of level areas
Plot the summer-stage of each level area.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot(color="k", ax=ax)
norm = matplotlib.colors.Normalize(vmin=-3, vmax=1)
cmap = "viridis"
for wb in la:
    la[wb].plot("summer_stage", ax=ax, norm=norm, cmap=cmap, zorder=0)
nlmod.plot.colorbar_inside(norm=norm, cmap=cmap);

Exercise 4: plot the difference between summer stage and winter stage.

## Assign stages from level areas to BGT-data
We then need to assign the summer- and winter-stages from the level-areas to the polygon-data of the surface water. We use the method `nlmod.gwf.surface_water.add_stages_from_waterboards()`.

In [ ]:
bgt = nlmod.gwf.surface_water.add_stages_from_waterboards(bgt, la=la)

## Plot summer stage of surface water features
Plot the summer stage of the surface water features.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
norm = matplotlib.colors.Normalize(vmin=-3, vmax=1)
cmap = "viridis"
bgt.plot("summer_stage", ax=ax, norm=norm, cmap=cmap)
nlmod.plot.colorbar_inside(norm=norm, cmap=cmap);

## Missing features
Determine how many polygons do not contain a valid summer stage

In [ ]:
n_nan = bgt['summer_stage'].isna().sum()
logger.warning(f'There are {n_nan} features without a summer stage')

## Assign stage of river Lek
The most important missing stage is of the river Lek. This river is connceted to the North Sea without any obstructions, so let's assign a fixed level of 0.0 m NAP to this river. You can select the river by determining where "bronhouder" is equal to "L0002", which is Rijkswaterstaat (the national water authority).

In [ ]:
mask = bgt["bronhouder"] == "L0002"
bgt.loc[mask, ['summer_stage', 'winter_stage']] = 0.0

Exercise 4: again plot the summer stage of the surface water features. If everything went right, the river Lek now also has a non-NaN stage.

Exercise 5: again determine how many features do not contain a summer-stage. These features will be ignored when using these river features in a model using `nlmod'.